## B. CellphoneDB:  Identify candidate cell-cell interactions

In [1]:
library(dplyr)
library(reshape2)
library(RColorBrewer)
library(harmony)
library(ggplot2)
library(pheatmap)
library(cowplot)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: Rcpp



In [2]:
# Define variables
filter_int_user_curated = T #Use only user_curated interactions?
per_cutoff = 0.1 # min % of cells in the cluster required  with expression > 0 for the gene
pval_cutoff = 0.001 # max adjusted p-value requeired to consider a gene as DEG
logFC_cutoff = 0.2 # min logFC to consider a gene as DEG
strict_DEG = F # Boolean, True if both partner in an interactions should be differentially expressed. False, if only one partner gene is requires to be a DEGs 

In [3]:
# Load cellphone database - genes and complexes file
# Gene names
genes_cpDB = read.delim('~/gonads/main/cellphoneDB/hsa_uniprot.txt', stringsAsFactors = F, sep = '\t')

# Complexes members
com_cpDB = read.csv('~/gonads/main/cellphoneDB/database/complex_curated_032021.tsv', stringsAsFactors = F, sep='\t')
com_cpDB = unique(com_cpDB)
com_cpDB$complex_name = paste0('complex:', com_cpDB$complex_name)
# Generate complexes2gene symbol dictionary
Com2Gene = lapply(com_cpDB$complex_name, function(cx) subset(genes_cpDB, uniprot %in% unlist(com_cpDB[com_cpDB$complex_name == cx, 2:5]) )$gene_name )
Com2Gene = lapply(Com2Gene, unique)
names(Com2Gene) = com_cpDB$complex_name
                  
# Load interactions from cellphoneDB/out/means.txt output file                   
int_cpDB = read.delim('~/gonads/main/cellphoneDB/out/means.txt', stringsAsFactors = F)[, 1:10]
# MANDATORY: remove "curated" because we have cleaned and renamed them (this is a long story, just do it)
int_cpDB = subset(int_cpDB, annotation_strategy != 'curated')
# OPTIONAL: Use only user_curated interactions?
if(filter_int_user_curated)
    int_cpDB = subset(int_cpDB, annotation_strategy == 'user_curated')
# Generate Int2Gene dictionary
Int2Gene = apply(int_cpDB, 1, function(int) {
    int = unname(int)
    if(int[5] == '') # if partnerA is complex, then retrieve members from dictionary
       partnerA = Com2Gene[[int[3]]]
    else  # if partnerA is a single gene, take name from file
       partnerA = int[5]
    if(int[6] == '')  # if partnerB is complex, then retrieve members from dictionary
       partnerB = Com2Gene[[int[4]]]
    else  # if partnerB is a single gene, take name from file
       partnerB = int[6]
    list(partner_a=partnerA,partner_b=partnerB)
    }) 
names(Int2Gene) = int_cpDB$interacting_pair

In [4]:
# Load somatic cells DEGs
# you need to put in a folder a file for each cluster that you included in cellphone
# The file must contain at least three columns (aka, you can reuse the output of the DEG files):
# - Gene: gene symbol
# - cluster: cell type
# - percentExpr_cluster: % of cells in the cluster with expression > 0 for the gene
path_Exp = 'cellphoneDB/input/percent.csv'


# Load
Per_df = read.csv(path_Exp, stringsAsFactors = F, row.names = 1) 
Per_df = setNames(melt(as.matrix(Per_df), factorsAsStrings = T), c('gene', 'cluster', 'percent'))


# Filetr expressed
Per_df = subset(Per_df, percent >= per_cutoff)
Per_df = Per_df[ order(Per_df$gene), ]

# Build dictionary clusters2expressed genes
clusters = unique(Per_df$cluster)
is_Exp = lapply(clusters, function(cl){
    filter(Per_df, cluster == cl)$gene
})
names(is_Exp) = clusters
sapply(is_Exp, length)
names(is_Exp) = gsub('pre.spermatogonia', 'pre-spermatogonia', names(is_Exp))

PGC       PGC_mitotic           coelEpi            oocyte 
             9634             10265              7472              8315 
    oogonia_STRA8   oogonia_meiosis pre.spermatogonia            sPAX8m 
            10418              7305             10300              6410 
          Sertoli     Sertoli_WFDC2       ovarianSurf          preGC_II 
             6271              8278              8223              7515 
        preGC_III   preGC_III_Notch  preGC_II_hypoxia      preGC_I_OSR1 
             7718              7876              8572              8197 
           sKITLG      sLGR5_female        sLGR5_male            sPAX8b 
             7937              8198              7808              6533

In [5]:
# load DE expression info
# you need to put in a folder the DE files of the clusters that you're interested in
# The file must contain at least four columns (aka, you can reuse the output of the DEG files):
# - Gene: gene symbol
# - cluster: cell type
# - adj.P.Val: adjusted p-value from the DE test
# - logFC: from the DE test
path_DEGs = 'cellphoneDB/input/DEGs/merged.csv'


# Load
DE_df = read.csv(path_DEGs, stringsAsFactors = F) %>%
        filter(., gene %in% genes_cpDB$gene_name) %>%
        filter(., p_val_adj < pval_cutoff) %>%
        filter(., avg_logFC > logFC_cutoff)
        


# Build dictionary clusters2expressed genes
clusters = unique(Per_df$cluster)
is_DE = lapply(clusters, function(cl){
    filter(DE_df, cluster == cl)$gene
})
names(is_DE) = clusters
  
sapply(is_DE, length)
names(is_DE) = gsub('pre.spermatogonia', 'pre-spermatogonia', names(is_DE))

PGC       PGC_mitotic           coelEpi            oocyte 
              552               541               511               740 
    oogonia_STRA8   oogonia_meiosis pre.spermatogonia            sPAX8m 
              573               445                 0               363 
          Sertoli     Sertoli_WFDC2       ovarianSurf          preGC_II 
             1105               686               344               181 
        preGC_III   preGC_III_Notch  preGC_II_hypoxia      preGC_I_OSR1 
              396               798               261               370 
           sKITLG      sLGR5_female        sLGR5_male            sPAX8b 
              244               432               669               514

In [6]:
# Get all pairwise cluster combinations: A--B, B--A
cluster_combinations = combn(names(is_Exp), 2, simplify = T)
ncol(cluster_combinations)

# add self interactions
cluster_combinations = cbind(sapply(names(is_Exp), function(x) rep(x, 2) ), cluster_combinations)


# We only want to test pairs including at least one cluster in the DE folder (here, epithelial)
keep = apply(cluster_combinations, 2, function(x) any(x %in% names(is_DE)))
cluster_combinations = cluster_combinations[, keep]
             
# # We only want to test pairs including at least one cluster named 'media'
# keep = apply(cluster_combinations, 2, function(x) any(x %in% 'Media'))
# cluster_combinations = cluster_combinations[, keep]
# ncol(cluster_combinations)

# Consider the reverse interaction too: A--B but B--A as well
cluster_combinations = cbind(cluster_combinations, cluster_combinations[2:1,]) %>% t(.) %>% unique(.)  %>% t(.) 

[1] 190

In [7]:
micro_df = read.csv('cellphoneDB/input/microenviroments/microenviroments.csv', stringsAsFactors = F)
microenviroments = split(micro_df, micro_df$microenviroment)  %>%  sapply(., select, celltype)

# Filter interactions within microenviroments
within_idx = c()
for (mienv in microenviroments){
    x = apply(cluster_combinations, 1, function(x) x %in% mienv) 
    within_idx = c(within_idx,
                   which(x[, 1] + x[, 2] == 2))
}
cluster_combinations = cluster_combinations[, within_idx]
              

In [8]:
# Make cluster pair labels: celltypeA--celltypeB             
cluster_combinations_labels = apply(cluster_combinations, 2, paste, collapse = '---')
ncol(cluster_combinations)
             
colnames(cluster_combinations) = cluster_combinations_labels

[1] 162

In [9]:
# Make scaffold matrix: L/R interactions (rows) x celltype pairs (columns)
E = matrix(0, nrow = length(Int2Gene), ncol = length(cluster_combinations_labels), 
           dimnames = list(names(Int2Gene), cluster_combinations_labels) )
dim(E) # is it big?

[1] 1674  162

In [10]:
# E will be a binary matrix with 1 indicating that all the genes in the interaction are expressed in the corresponding celltype
# So, fill 1 if all genes are expressed in all clusters
for (Int in rownames(E)){ # iterate over interactions
    for (clpair in colnames(E)){ # iterate over celltype pairs
        cl_A = cluster_combinations[1, clpair] # celltype_A
        cl_B = cluster_combinations[2, clpair] # celltype_B
        are_all_expressed = all(Int2Gene[[Int]]$partner_a %in% is_Exp[[cl_A]]) &&  # are all partner_A genes expressed in celltype_A
                            all(Int2Gene[[Int]]$partner_b %in% is_Exp[[cl_B]])     # are all partner_B genes expressed in celltype_B
        if(are_all_expressed)
            E[Int, clpair] = 1
    }
}

In [11]:
# keep celltypepairs with at least one expressed interaction
keep = apply(E, 2, sum) > 0
E = E[,keep]
# is it big?
dim(E)
sum(E)

[1] 1674  158

[1] 9019

In [12]:
# keep interactions expressed in at least one celltype pair
keep = apply(E, 1, sum) > 0
E = E[keep,]
# is it big?
dim(E)
sum(E)

[1] 464 158

[1] 9019

In [13]:
# Initialize DE matrix from E and set all values to 0
# DE will be a binary matrix with 1 indicating that all the genes in the interaction are expressed in the corresponding celltype
# and one is a DE in the celltypes of interests
DE = E
DE[] = 0

In [14]:
if (strict_DEG){
    # Fill 1 if BOTH GENES in the interaction is DE
    for (Int in rownames(DE)){ # iterate over interactions
        for (clpair in colnames(DE)){ # iterate over celltype pairs
            cl_A = cluster_combinations[1, clpair] # celltype_A
            cl_B = cluster_combinations[2, clpair] # celltype_B
            any_DE = any(Int2Gene[[Int]]$partner_a %in% is_DE[[cl_A]]) &&  # is any partner_A gene DE in celltype_A
                     any(Int2Gene[[Int]]$partner_b %in% is_DE[[cl_B]])     # is any partner_B gene DE in celltype_B
            if(any_DE & E[Int, clpair] == 1) # check all members are expressed
                DE[Int, clpair] = 1
        }
    }
}else{
    #Fill 1 if at least one gene in the interaction is DE
    for (Int in rownames(DE)){ # iterate over interactions
        for (clpair in colnames(DE)){ # iterate over celltype pairs
            cl_A = cluster_combinations[1, clpair] # celltype_A
            cl_B = cluster_combinations[2, clpair] # celltype_B
            any_DE = any(Int2Gene[[Int]]$partner_a %in% is_DE[[cl_A]]) ||  # is any partner_A gene DE in celltype_A
                     any(Int2Gene[[Int]]$partner_b %in% is_DE[[cl_B]])     # is any partner_B gene DE in celltype_B
            if(any_DE & E[Int, clpair] == 1) # check all members are expressed
                DE[Int, clpair] = 1
        }
    }
}

In [15]:
# keep celltypepairs with at least one expressed interaction
keep = apply(DE, 2, sum) > 0
DE = DE[,keep]

# keep interactions expressed in at least one celltype pair
keep = apply(DE, 1, sum) > 0
DE = DE[keep,]

# is it big?
dim(DE)
sum(DE)


[1] 377 157

[1] 3414

In [16]:
write.csv(DE, file = paste0('cellphoneDB/out/interactions_' , pval_cutoff, '_logFC', logFC_cutoff, '.csv'))

In [17]:
rownames(DE)[ grep('RAr', rownames(DE)) ]



[1] "RAreceptor_RARA_atRetinoicAcid_byALDH1A1"     
 [2] "RAreceptor_RARA_RXRA_atRetinoicAcid_byALDH1A1"
 [3] "RAreceptor_RARB_atRetinoicAcid_byALDH1A1"     
 [4] "RAreceptor_RARB_RXRB_atRetinoicAcid_byALDH1A1"
 [5] "RAreceptor_RARG_atRetinoicAcid_byALDH1A1"     
 [6] "RAreceptor_RXRA_atRetinoicAcid_byALDH1A1"     
 [7] "RAreceptor_RXRB_atRetinoicAcid_byALDH1A1"     
 [8] "RAreceptor_RXRG_atRetinoicAcid_byALDH1A1"     
 [9] "RAreceptor_RARA_atRetinoicAcid_byALDH1A2"     
[10] "RAreceptor_RARA_RXRA_atRetinoicAcid_byALDH1A2"
[11] "RAreceptor_RARB_atRetinoicAcid_byALDH1A2"     
[12] "RAreceptor_RARB_RXRB_atRetinoicAcid_byALDH1A2"
[13] "RAreceptor_RARG_atRetinoicAcid_byALDH1A2"     
[14] "RAreceptor_RXRA_atRetinoicAcid_byALDH1A2"     
[15] "RAreceptor_RXRB_atRetinoicAcid_byALDH1A2"     
[16] "RAreceptor_RXRG_atRetinoicAcid_byALDH1A2"

In [18]:
colnames(DE)

[1] "PGC---PGC"                          
  [2] "PGC_mitotic---PGC_mitotic"          
  [3] "coelEpi---coelEpi"                  
  [4] "ovarianSurf---ovarianSurf"          
  [5] "preGC_II---preGC_II"                
  [6] "preGC_II_hypoxia---preGC_II_hypoxia"
  [7] "sKITLG---sKITLG"                    
  [8] "sLGR5_female---sLGR5_female"        
  [9] "PGC---PGC_mitotic"                  
 [10] "PGC---coelEpi"                      
 [11] "PGC---ovarianSurf"                  
 [12] "PGC---preGC_II"                     
 [13] "PGC---preGC_II_hypoxia"             
 [14] "PGC---sKITLG"                       
 [15] "PGC---sLGR5_female"                 
 [16] "PGC_mitotic---coelEpi"              
 [17] "PGC_mitotic---ovarianSurf"          
 [18] "PGC_mitotic---preGC_II"             
 [19] "PGC_mitotic---preGC_II_hypoxia"     
 [20] "PGC_mitotic---sKITLG"               
 [21] "PGC_mitotic---sLGR5_female"         
 [22] "coelEpi---ovarianSurf"              
 [23] "coelEpi---preGC_II"                 
 [24] "coelEpi---preGC_II_hypoxia"         
 [25] "coelEpi---sKITLG"                   
 [26] "coelEpi---sLGR5_female"             
 [27] "ovarianSurf---preGC_II"             
 [28] "ovarianSurf---preGC_II_hypoxia"     
 [29] "ovarianSurf---sKITLG"               
 [30] "ovarianSurf---sLGR5_female"         
 [31] "preGC_II---preGC_II_hypoxia"        
 [32] "preGC_II---sKITLG"                  
 [33] "preGC_II---sLGR5_female"            
 [34] "preGC_II_hypoxia---sKITLG"          
 [35] "preGC_II_hypoxia---sLGR5_female"    
 [36] "sKITLG---sLGR5_female"              
 [37] "PGC_mitotic---PGC"                  
 [38] "coelEpi---PGC"                      
 [39] "ovarianSurf---PGC"                  
 [40] "preGC_II---PGC"                     
 [41] "preGC_II_hypoxia---PGC"             
 [42] "sKITLG---PGC"                       
 [43] "sLGR5_female---PGC"                 
 [44] "coelEpi---PGC_mitotic"              
 [45] "ovarianSurf---PGC_mitotic"          
 [46] "preGC_II---PGC_mitotic"             
 [47] "preGC_II_hypoxia---PGC_mitotic"     
 [48] "sKITLG---PGC_mitotic"               
 [49] "sLGR5_female---PGC_mitotic"         
 [50] "ovarianSurf---coelEpi"              
 [51] "preGC_II---coelEpi"                 
 [52] "preGC_II_hypoxia---coelEpi"         
 [53] "sKITLG---coelEpi"                   
 [54] "sLGR5_female---coelEpi"             
 [55] "preGC_II---ovarianSurf"             
 [56] "preGC_II_hypoxia---ovarianSurf"     
 [57] "sKITLG---ovarianSurf"               
 [58] "sLGR5_female---ovarianSurf"         
 [59] "preGC_II_hypoxia---preGC_II"        
 [60] "sKITLG---preGC_II"                  
 [61] "sLGR5_female---preGC_II"            
 [62] "sKITLG---preGC_II_hypoxia"          
 [63] "sLGR5_female---preGC_II_hypoxia"    
 [64] "sLGR5_female---sKITLG"              
 [65] "oocyte---oocyte"                    
 [66] "oogonia_STRA8---oogonia_STRA8"      
 [67] "oogonia_meiosis---oogonia_meiosis"  
 [68] "preGC_III---preGC_III"              
 [69] "preGC_III_Notch---preGC_III_Notch"  
 [70] "preGC_I_OSR1---preGC_I_OSR1"        
 [71] "sPAX8b---sPAX8b"                    
 [72] "oocyte---oogonia_STRA8"             
 [73] "oocyte---oogonia_meiosis"           
 [74] "oocyte---preGC_III"                 
 [75] "oocyte---preGC_III_Notch"           
 [76] "oocyte---preGC_I_OSR1"              
 [77] "oocyte---sPAX8b"                    
 [78] "oogonia_STRA8---oogonia_meiosis"    
 [79] "oogonia_STRA8---preGC_III"          
 [80] "oogonia_STRA8---preGC_III_Notch"    
 [81] "oogonia_STRA8---preGC_I_OSR1"       
 [82] "oogonia_STRA8---sPAX8b"             
 [83] "oogonia_meiosis---preGC_III"        
 [84] "oogonia_meiosis---preGC_III_Notch"  
 [85] "oogonia_meiosis---preGC_I_OSR1"     
 [86] "oogonia_meiosis---sPAX8b"           
 [87] "preGC_III---preGC_III_Notch"        
 [88] "preGC_III---preGC_I_OSR1"           
 [89] "preGC_III---sPAX8b"                 
 [90] "preGC_III_Notch---preGC_I_OSR1"     
 [91] "preGC_III_Notch---sPAX8b"          